# **Polars: De Conceptos a Aplicaciones Prácticas**

## Objetivos del Curso
- Comprender qué es Polars y por qué es la evolución de Pandas
- Dominar los conceptos de lazy evaluation y optimización de queries
- Procesar datasets grandes con máxima eficiencia
- Aplicar Polars en casos prácticos reales
- Dominar expresiones y query optimization

## Estructura del Curso
1. Introducción y Fundamentos (15 min) - Qué es Polars?
2. Conceptos Técnicos (20 min) - Lazy Evaluation y Expresiones
3. Comparación Práctica (15 min) - Pandas vs Polars
4. Caso Práctico (20 min) - Dataset de Vuelos Real
5. Optimización Avanzada (10 min) - Mejores Prácticas


## 1. INTRODUCCIÓN: QUÉ ES POLARS?

Polars es una biblioteca de DataFrame rápida y eficiente en memoria escrita en Rust, con bindings de Python. Está diseñada para procesar datos masivos con máximo rendimiento y paralelización automática.

### Qué problemas resuelve Polars?

- Velocidad: Procesamiento hasta 10x más rápido que Pandas
- Memoria: Uso eficiente de RAM con streaming y evaluación perezosa
- Escalabilidad: Desde laptop hasta clusters distribuidos
- API moderna: Sintaxis intuitiva con query optimization
- Paralelización automática: Aprovecha múltiples cores sin configuración

### Ventajas principales:
- Velocidad extrema: Implementado en Rust, optimizada desde cero
- Lazy evaluation: Optimización de queries antes de ejecución
- Expresiones: API declarativa para transformaciones complejas
- Streaming: Procesa datasets más grandes que la memoria disponible
- Zero-copy: Minimiza copias de datos innecesarias
- Polars SQL: Consulta datos con sintaxis SQL


### Instalación

Si aún no tienes instalado polars, ejecuta:

In [ ]:
# ! pip install polars

Para funcionalidades completas, instala con dependencias opcionales:

In [ ]:
# ! pip install polars[all]

### Comparación: Polars vs Pandas

| Característica | Pandas | Polars |
|----------------|--------|--------|
| Velocidad | Media (optimizada en Python) | Alta (Rust, optimizada) |
| Memoria | Menos eficiente | Muy eficiente (zero-copy) |
| Lazy Evaluation | No (extensiones como Dask) | Sí (nativo) |
| Paralelización | Limitada | Automática y completa |
| API | Imperativa | Declarativa (expresiones) |
| Streaming | No | Sí (out-of-core) |
| SQL Support | Sí (con sql_query) | Sí (nativo con SQL) |


## 2. IMPORTACIÓN Y CONFIGURACIÓN

In [ ]:
# Importar las librerías necesarias
import polars as pl
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

print(f"Versión de Polars: {pl.__version__}")
print(f"Versión de Pandas: {pd.__version__}")
print(f"Versión de NumPy: {np.__version__}")
print("Imports completados exitosamente")

# Configuración para mejor visualización
pl.Config.set_tbl_formatting("UTF8_FULL")
pl.Config.set_tbl_hide_column_data_types(True)

## 3. CONCEPTOS FUNDAMENTALES

### A. DataFrame y Series

Polars usa estructuras de datos similares a Pandas pero con mayor rendimiento:
- **DataFrame**: Estructura tabular bidimensional
- **Series**: Estructura unidimensional
- **Expresiones**: Transformaciones declarativas de datos


In [ ]:
# Creación básica de DataFrames
print("CREACIÓN DE DATAFRAMES BÁSICOS")
print("=" * 40)

# Método 1: A partir de diccionario
df_basico = pl.DataFrame({
    'nombre': ['Ana', 'Luis', 'María', 'Carlos'],
    'edad': [25, 30, 35, 28],
    'ciudad': ['Madrid', 'Barcelona', 'Valencia', 'Sevilla']
})

print("DataFrame básico:")
print(df_basico)

# Método 2: A partir de listas
df_listas = pl.DataFrame({
    'producto': ['Laptop', 'Mouse', 'Teclado', 'Monitor'],
    'precio': [999.99, 25.50, 75.25, 299.99],
    'stock': [10, 150, 75, 25]
})

print("\nDataFrame de productos:")
print(df_listas)

In [ ]:
# Información básica del DataFrame
print("INFORMACIÓN DEL DATAFRAME")
print("=" * 35)

print(f"Forma: {df_basico.shape}")
print(f"Columnas: {df_basico.columns}")
print(f"Tipos de datos:")
print(df_basico.dtypes)

print("\nDescripción estadística:")
print(df_basico.describe())

### B. Lazy Evaluation (Evaluación Perezosa)

Concepto clave: Polars no ejecuta operaciones inmediatamente en lazy mode, las optimiza en un plan de ejecución que se ejecuta solo cuando:
- Llamamos `.collect()`
- Guardamos a disco
- Mostramos resultados (trigger implícito)

Esta estrategia permite:
- Optimización global del plan de ejecución
- Paralelización inteligente de operaciones
- Mínimo uso de memoria


In [ ]:
# Demostración de Lazy Evaluation
print("DEMOSTRACIÓN DE LAZY EVALUATION")
print("=" * 40)

# Crear DataFrame en modo eager
df_lazy = pl.DataFrame({
    'id': range(100000),
    'valor': np.random.randn(100000),
    'categoria': np.random.choice(['A', 'B', 'C'], 100000),
    'grupo': np.random.choice(['X', 'Y', 'Z'], 100000)
})

# Cambiar a modo lazy
df_lazy = df_lazy.lazy()

print(f"DataFrame lazy creado: {type(df_lazy)}")
print("\nPlan de ejecución (SIN ejecutar):")
print(df_lazy)

In [ ]:
# Ejecutar operaciones lazy
print("EJECUTANDO OPERACIONES LAZY")
print("=" * 35)

start_time = time.time()

# Operaciones encadenadas (se optimizan todas juntas)
resultado = (
    df_lazy
    .filter(pl.col('valor') > 0)  # Filtrar valores positivos
    .group_by('categoria')
    .agg([
        pl.col('valor').mean().alias('promedio'),
        pl.col('valor').count().alias('conteo')
    ])
    .sort('promedio', descending=True)
).collect()  # ¡Aquí se ejecuta todo!

execution_time = time.time() - start_time

print(f"Ejecutado en {execution_time:.3f} segundos")
print("\nResultado optimizado:")
print(resultado)

## 4. CASO PRÁCTICO: DATASET DE VUELOS

### Análisis de Datos de Aviación con Polars

Usaremos un dataset real de vuelos para demostrar el poder de Polars:

In [ ]:
# Configuración de dataset
print("CONFIGURACIÓN DE DATOS DE VUELOS")
print("=" * 40)

# URLs de datasets de vuelos
small_csv = 'https://github.com/ricardoahumada/data-for-auditors/raw/refs/heads/main/4.%20Análisis%20Masivo%20de%20Datos/Optimizacion/data/2008_very_small.csv'

print("Dataset disponible: vuelos muy pequeños (~7MB) para demo")
print("Polars puede manejar datasets mucho más grandes eficientemente")

In [ ]:
# Cargar dataset con Polars
print("Cargando dataset con Polars...")
start_time = time.time()

try:
    # Cargar con Polars (más rápido que pandas)
    df_vuelos = pl.read_csv(small_csv)
    load_time = time.time() - start_time
    
    print(f"Dataset cargado en {load_time:.2f} segundos")
    print(f"Forma: {df_vuelos.shape}")
    print(f"Columnas: {df_vuelos.columns}")
    
except Exception as e:
    print(f"Error cargando dataset: {e}")
    print("Generando dataset sintético de vuelos...")
    
    # Dataset sintético alternativo
    np.random.seed(42)
    n_filas = 50000
    
    df_vuelos = pl.DataFrame({
        'Year': np.random.randint(2008, 2009, n_filas),
        'Month': np.random.randint(1, 13, n_filas),
        'DayofMonth': np.random.randint(1, 32, n_filas),
        'DayOfWeek': np.random.randint(1, 8, n_filas),
        'DepDelay': np.random.randint(-30, 300, n_filas),
        'ArrDelay': np.random.randint(-30, 300, n_filas),
        'Origin': np.random.choice(['JFK', 'LAX', 'ORD', 'ATL', 'DFW', 'SFO'], n_filas),
        'Dest': np.random.choice(['JFK', 'LAX', 'ORD', 'ATL', 'DFW', 'SFO'], n_filas),
        'Distance': np.random.randint(100, 3000, n_filas)
    })
    print("Dataset sintético de vuelos creado")

print(f"\nDataset final: {df_vuelos.shape}")

In [ ]:
# Verificar estructura del DataFrame
print("ESTRUCTURA DEL DATASET DE VUELOS")
print("=" * 40)

print("\nPrimeras 5 filas:")
print(df_vuelos.head())

print("\nTipos de datos:")
print(df_vuelos.dtypes)

print("\nInformación estadística básica:")
print(df_vuelos.describe())

### Análisis de Vuelos con Polars

In [ ]:
# Análisis de retrasos de vuelos
print("ANÁLISIS DE VUELOS CON POLARS")
print("=" * 40)

# 1. Filtrar vuelos con retraso
print("\n1. Vuelos con retraso de salida > 15 minutos:")
start = time.time()

vuelos_retrasados = df_vuelos.filter(pl.col('DepDelay') > 15)
count_retrasados = vuelos_retrasados.shape[0]
total_vuelos = df_vuelos.shape[0]
porcentaje = (count_retrasados / total_vuelos) * 100

print(f"Tiempo: {time.time() - start:.3f} segundos")
print(f"Vuelos con retraso: {count_retrasados:,} de {total_vuelos:,} total")
print(f"Porcentaje: {porcentaje:.1f}%")

# 2. Análisis por aeropuerto de origen
print("\n2. Análisis por aeropuerto de origen:")
retrasos_origen = df_vuelos.group_by('Origin').agg([
    pl.col('DepDelay').mean().alias('retraso_promedio'),
    pl.col('DepDelay').count().alias('total_vuelos')
]).sort('retraso_promedio', descending=True)

print(retrasos_origen)

In [ ]:
# 3. Calcular estadísticas complejas
print("\n3. Análisis temporal por mes:")

# Agregar columna de mes si no existe
if 'Month' in df_vuelos.columns:
    analisis_mensual = df_vuelos.group_by('Month').agg([
        pl.col('DepDelay').mean().alias('retraso_promedio'),
        pl.col('DepDelay').max().alias('retraso_maximo'),
        pl.col('DepDelay').count().alias('vuelos_totales')
    ]).sort('Month')
    
    print(analisis_mensual)

# 4. Análisis de correlaciones
print("\n4. Análisis de correlación:")
if 'DepDelay' in df_vuelos.columns and 'ArrDelay' in df_vuelos.columns:
    correlacion = df_vuelos.select(pl.corr(pl.col('DepDelay'), pl.col('ArrDelay')).alias('correlacion'))
    print(f"Correlación entre retraso de salida y llegada: {correlacion['correlacion'][0]:.3f}")

# 5. Top destinos con más vuelos
print("\n5. Top 5 destinos por número de vuelos:")
top_destinos = df_vuelos.group_by('Dest').agg([
    pl.col('Dest').count().alias('vuelos')
]).sort('vuelos', descending=True).head(5)

print(top_destinos)

## 5. COMPARACIÓN PRÁCTICA: PANDAS VS POLARS

### Performance en Dataset Real

Vamos a comparar el rendimiento entre Pandas y Polars:

In [ ]:
# Comparación directa Pandas vs Polars
print("COMPARACIÓN DE PERFORMANCE: PANDAS VS POLARS")
print("=" * 55)

# Convertir a pandas para comparación
print("Convirtiendo a pandas para comparación...")
df_pandas = df_vuelos.to_pandas()
print(f"Dataset pandas: {df_pandas.shape}")

print("\nComparación de operaciones básicas:")

# Operación 1: Filtrado
print("\n1. FILTRADO: Vuelos con DepDelay > 15")

# Pandas
start = time.time()
pandas_filtro = df_pandas[df_pandas['DepDelay'] > 15]
pandas_time = time.time() - start
pandas_result_count = len(pandas_filtro)

# Polars
start = time.time()
polars_filtro = df_vuelos.filter(pl.col('DepDelay') > 15)
polars_time = time.time() - start
polars_result_count = len(polars_filtro)

print(f"   Pandas: {pandas_time:.4f} segundos - {pandas_result_count:,} resultados")
print(f"   Polars: {polars_time:.4f} segundos - {polars_result_count:,} resultados")

ganador = "Polars" if polars_time < pandas_time else "Pandas"
ratio = min(pandas_time, polars_time) / max(pandas_time, polars_time)
print(f"   Ganador: {ganador} (x{1/ratio:.1f} más rápido)")

In [ ]:
# Operación 2: GroupBy complejo
print("\n2. GROUPBY: Análisis por aeropuerto de origen")

# Pandas
start = time.time()
pandas_groupby = df_pandas.groupby('Origin').agg({
    'DepDelay': ['mean', 'count', 'max'],
    'ArrDelay': 'mean'
})
pandas_time = time.time() - start

# Polars
start = time.time()
polars_groupby = df_vuelos.group_by('Origin').agg([
    pl.col('DepDelay').mean().alias('dep_delay_mean'),
    pl.col('DepDelay').count().alias('dep_delay_count'),
    pl.col('DepDelay').max().alias('dep_delay_max'),
    pl.col('ArrDelay').mean().alias('arr_delay_mean')
])
polars_time = time.time() - start

print(f"   Pandas: {pandas_time:.4f} segundos")
print(f"   Polars: {polars_time:.4f} segundos")

ganador = "Polars" if polars_time < pandas_time else "Pandas"
ratio = min(pandas_time, polars_time) / max(pandas_time, polars_time)
print(f"   Ganador: {ganador} (x{1/ratio:.1f} más rápido)")

## 6. OPTIMIZACIÓN AVANZADA

### Técnicas para Maximizar el Rendimiento de Polars

In [ ]:
# Optimización 1: Lazy evaluation
print("OPTIMIZACIÓN 1: LAZY EVALUATION")
print("=" * 40)

print("Demostrando optimización con lazy mode:")

# Versión eager (inmediata)
print("\nVersión Eager (inmediata):")
start = time.time()
result_eager = (
    df_vuelos
    .filter(pl.col('DepDelay') > 0)
    .group_by('Origin')
    .agg(pl.col('DepDelay').mean().alias('promedio'))
    .sort('promedio', descending=True)
)
eager_time = time.time() - start
print(f"Tiempo: {eager_time:.3f} segundos")

# Versión lazy (optimizada)
print("\nVersión Lazy (optimizada):")
start = time.time()
result_lazy = (
    df_vuelos
    .lazy()
    .filter(pl.col('DepDelay') > 0)
    .group_by('Origin')
    .agg(pl.col('DepDelay').mean().alias('promedio'))
    .sort('promedio', descending=True)
    .collect()  # Solo aquí se ejecuta
)
lazy_time = time.time() - start
print(f"Tiempo: {lazy_time:.3f} segundos")

print(f"\nLazy evaluation fue {eager_time/lazy_time:.1f}x más rápido" if lazy_time < eager_time else "Eager fue más rápido en este caso")

In [ ]:
# Optimización 2: Tipos de datos
print("\nOPTIMIZACIÓN 2: TIPOS DE DATOS EFICIENTES")
print("=" * 45)

print("\nTipos de datos originales:")
print(df_vuelos.dtypes)

# Optimizar tipos de datos
print("\nOptimizando tipos de datos...")

# Convertir a lazy para mejor optimización
df_optimized = df_vuelos.lazy()

# Usar cast para optimizar tipos
df_optimized = df_optimized.with_columns([
    pl.col('Year').cast(pl.Int16),
    pl.col('Month').cast(pl.Int8),
    pl.col('DayofMonth').cast(pl.Int8),
    pl.col('DayOfWeek').cast(pl.Int8),
    pl.col('DepDelay').cast(pl.Int16),
    pl.col('ArrDelay').cast(pl.Int16),
    pl.col('Distance').cast(pl.Int16)
])

# Convertir strings a categorías para ahorro de memoria
if 'Origin' in df_optimized.columns:
    df_optimized = df_optimized.with_columns(
        pl.col('Origin').cast(pl.Categorical)
    )

if 'Dest' in df_optimized.columns:
    df_optimized = df_optimized.with_columns(
        pl.col('Dest').cast(pl.Categorical)
    )

# Recolectar resultado optimizado
df_final = df_optimized.collect()

print("Tipos de datos optimizados:")
print(df_final.dtypes)

## 7. EXPRESIONES AVANZADAS Y SQL

### Query Optimization con Expresiones

In [ ]:
# Expresiones complejas con Polars
print("EXPRESIONES AVANZADAS")
print("=" * 30)

# Análisis complejo en una sola query
analisis_complejo = df_vuelos.select([
    'Origin',
    'DepDelay',
    'ArrDelay',
    'Distance'
]).with_columns([
    # Crear indicadores binarios
    (pl.col('DepDelay') > 15).alias('retraso_significativo'),
    (pl.col('Distance') > 1000).alias('vuelo_largo'),
    
    # Categorizar retrasos
    pl.when(pl.col('DepDelay') < 0)
    .then(pl.lit('Temprano'))
    .when(pl.col('DepDelay') <= 15)
    .then(pl.lit('A tiempo'))
    .otherwise(pl.lit('Retrasado'))
    .alias('categoria_retraso'),
    
    # Ratio de eficiencia
    (pl.col('ArrDelay') - pl.col('DepDelay')).alias('delta_retraso')
]).group_by(['Origin', 'categoria_retraso']).agg([
    pl.col('DepDelay').mean().alias('promedio_retraso'),
    pl.col('retraso_significativo').mean().alias('porcentaje_retrasos'),
    pl.col('vuelo_largo').mean().alias('porcentaje_vuelos_largos'),
    pl.col('delta_retraso').mean().alias('delta_promedio')
])

print("\nAnálisis complejo por aeropuerto y categoría:")
print(analisis_complejo)

In [ ]:
# SQL con Polars
print("\nUSO DE SQL EN POLARS")
print("=" * 30)

# Ejecutar query SQL directamente
query_sql = """
SELECT 
    Origin,
    AVG(DepDelay) as promedio_retraso,
    COUNT(*) as total_vuelos,
    AVG(CASE WHEN DepDelay > 15 THEN 1 ELSE 0 END) as tasa_retrasos
FROM self
WHERE DepDelay IS NOT NULL
GROUP BY Origin
ORDER BY promedio_retraso DESC
LIMIT 5
"""

resultado_sql = df_vuelos.sql(query_sql)

print("Top 5 aeropuertos con más retrasos (SQL):")
print(resultado_sql)

## 8. EJERCICIO FINAL: TU TURNO

### **Tiempo: 10 minutos**

**Objetivo**: Analizar patrones de vuelos usando Polars

**Dataset**: Usa el mismo `df_vuelos` que hemos estado trabajando

**Tareas**:
1. **Encuentra** el aeropuerto con más vuelos (por volumen)
2. **Calcula** el vuelo promedio más eficiente (menor diferencia entre salida y llegada)
3. **Identifica** patrones de retrasos por día de la semana
4. **Determina** qué aeropuerto tiene la mejor puntualidad promedio

**Instrucciones**:
- Usa expresiones de Polars para las transformaciones
- Utiliza .group_by() y .agg() para agregaciones
- Aplica .sort() para organizar resultados
- Muestra los resultados de forma clara
- Explica qué insights obtienes


In [ ]:
# EJERCICIO FINAL - Completa el código

print("EJERCICIO FINAL: Análisis Avanzado de Vuelos")
print("=" * 55)

# Tu código aquí:

# 1. AEROPUERTO CON MÁS VUELOS (por volumen)
print("\n1. AEROPUERTO CON MÁS VUELOS:")
# Hint: usa group_by('Origin') y count()




# 2. ANÁLISIS DE EFICIENCIA DE VUELOS
print("\n2. VUELOS MÁS EFICIENTES:")
# Hint: calcula la diferencia entre ArrDelay y DepDelay




# 3. PATRONES POR DÍA DE LA SEMANA
print("\n3. ANÁLISIS POR DÍA DE LA SEMANA:")
# Hint: agrupa por 'DayOfWeek' y calcula promedio de retrasos




# 4. AEROPUERTO MÁS PUNTUAL
print("\n4. AEROPUERTO MÁS PUNTUAL:")
# Hint: ordena por promedio de retrasos en orden ascendente




# 5. INSIGHTS ADICIONALES
print("\n5. INSIGHTS Y RECOMENDACIONES:")
# ¿Qué patrones interesante encuentras?
# ¿Qué recomendaciones harías a las aerolíneas?




print("\n¡Ejercicio completado!")

## 9. RESUMEN Y CASOS DE USO

### Lo que has aprendido en este curso:

1. **DataFrame y Series**: Estructuras de datos optimizadas de Polars
2. **Lazy Evaluation**: Optimización automática y paralelización
3. **Expresiones**: API declarativa para transformaciones complejas
4. **Performance**: Velocidad superior a Pandas en operaciones complejas
5. **Tipos de datos**: Optimización de memoria con cast() y categorías
6. **SQL nativo**: Consulta de datos con sintaxis SQL
7. **Casos prácticos**: Análisis real de vuelos con datasets grandes

### Casos de uso ideales para Polars:

- Datasets de 100MB+ en laptop personal
- Análisis de datos financieros de alta frecuencia
- Procesamiento ETL de archivos grandes
- Análisis científico con millones de registros
- Machine Learning con datasets grandes
- Dashboards interactivos con datos complejos
- ETL de streaming en tiempo real

### Cuándo usar Polars vs Pandas:

| Situación | Recomendación |
|-----------|---------------|
| Dataset < 50MB | Pandas (más simple) |
| Dataset 50MB - 500MB | Polars (mejor performance) |
| Dataset > 500MB | Polars (necesario para eficiencia) |
| Operaciones complejas con múltiples group_by | Polars |
| Necesitas SQL nativo | Polars |
| API familiar y simple | Pandas |
| Máxima performance requerida | Polars |


## 10. RECURSOS ADICIONALES

### Documentación y Aprendizaje

- Documentación oficial: https://pola.rs/
- Tutorial interactivo: https://pola.rs/learn/
- GitHub: https://github.com/pola-rs/polars
- Ejemplos y benchmarks: https://github.com/pola-rs/polars-examples
- SQL Guide: https://pola.rs/sql/

### Herramientas Relacionadas

- **Polars Cloud**: Análisis colaborativo en la nube
- **Polars SQL**: Motor SQL nativo para Polars
- **Polars-Rust**: Para integración en aplicaciones Rust
- **DataFusion**: Integración con Apache Arrow

### Próximos Pasos

1. **Practica** con tus propios datasets grandes
2. **Experimenta** con lazy vs eager evaluation
3. **Aprende** expresiones más complejas
4. **Explora** integración con machine learning
5. **Considera** Polars para proyectos de análisis profesional
